In [2]:
import findspark
findspark.init()

In [3]:
import pyspark

In [4]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [5]:
path = "../data/raw/wls_day-01"
df = spark.read.json(path)

In [6]:
df.show()

+---------------------+-----------+------------+-------+-------------+-------------------+---------+---------+--------------------+---------------+-----------------+---------+--------------+-----------+----------+------+-----------------+--------------+---------------+----+-----------+
|AuthenticationPackage|Destination|  DomainName|EventID|FailureReason|            LogHost|  LogonID|LogonType|LogonTypeDescription|ParentProcessID|ParentProcessName|ProcessID|   ProcessName|ServiceName|    Source|Status|SubjectDomainName|SubjectLogonID|SubjectUserName|Time|   UserName|
+---------------------+-----------+------------+-------+-------------+-------------------+---------+---------+--------------------+---------------+-----------------+---------+--------------+-----------+----------+------+-----------------+--------------+---------------+----+-----------+
|                 null|       null|   Domain001|   4688|         null|         Comp607982|    0x3e7|     null|                null|        

In [7]:
df.columns

['AuthenticationPackage',
 'Destination',
 'DomainName',
 'EventID',
 'FailureReason',
 'LogHost',
 'LogonID',
 'LogonType',
 'LogonTypeDescription',
 'ParentProcessID',
 'ParentProcessName',
 'ProcessID',
 'ProcessName',
 'ServiceName',
 'Source',
 'Status',
 'SubjectDomainName',
 'SubjectLogonID',
 'SubjectUserName',
 'Time',
 'UserName']

In [10]:
df.groupBy('EventID').count().show()

+-------+--------+
|EventID|   count|
+-------+--------+
|   4625|  335425|
|   4608|    1607|
|   4802|    5148|
|   4672| 4388232|
|   4770|      85|
|   4801|   14899|
|   1100|    1637|
|   4648|  895145|
|   4647|    3060|
|   4624|14110061|
|   4768|  696025|
|   4769| 1762893|
|   4688|14034308|
|   4634|13826193|
|   4689|    1111|
|   4800|   15917|
|   4803|    4833|
|   4609|       9|
|   4776| 5505769|
|   4774|    6794|
+-------+--------+



In [11]:
df.groupBy('UserName').count().show()

+-----------+-----+
|   UserName|count|
+-----------+-----+
|Comp334881$| 2327|
|Comp193344$| 1753|
|Comp626532$| 1696|
|Comp423822$| 1581|
|Comp071603$| 1716|
|Comp194392$| 1541|
|Comp079982$| 2467|
|Comp102246$| 1822|
|Comp924592$| 1855|
|Comp332130$| 5977|
|Comp629929$| 1149|
|Comp989334$| 1676|
|Comp851861$| 2293|
|Comp854936$| 1737|
|Comp282520$| 3297|
|Comp349824$| 2129|
|Comp156468$| 1536|
|Comp528070$| 1714|
|Comp811288$| 1865|
|Comp925148$| 1705|
+-----------+-----+
only showing top 20 rows



In [12]:
df.groupBy('LogonType').count().show()

+---------+--------+
|LogonType|   count|
+---------+--------+
|        0|    1616|
|        7|   73211|
|     null|27337472|
|        9|   24850|
|        5|  191974|
|       10|    2271|
|        3|27642694|
|        8|   89498|
|       11|     589|
|        2|  240504|
|        4|    4472|
+---------+--------+



In [13]:
df.groupBy('Source').count().show()

+----------+-----+
|    Source|count|
+----------+-----+
|Comp095019|  351|
|Comp361193|30688|
|Comp515833| 3941|
|Comp129210|  368|
|Comp265496| 1469|
|Comp793733| 5792|
|Comp213727|  774|
|Comp403467|  796|
|Comp852067|  360|
|Comp449379| 1039|
|Comp495040|  809|
|Comp893211|  857|
|Comp237460|  665|
|Comp795960| 1428|
|Comp044445|  571|
|Comp886769|  872|
|Comp189990| 1668|
|Comp879962|  458|
|Comp721320|  843|
|Comp507421| 1300|
+----------+-----+
only showing top 20 rows

